In [2]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

import logging
logging.basicConfig()

from pathlib import Path
from pprint import pprint

from tqdm import tqdm, trange

import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
from scipy import ndimage as ndi

import fish

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
NOTEBOOK_NAME = "pipette"

HERE = Path.cwd().absolute()
DATA = HERE.parent / "data"
OUT = HERE / "out" / Path(NOTEBOOK_NAME).stem
print(HERE)
print(DATA)
print(OUT)

D:\projects\fish\dev
D:\projects\fish\data
D:\projects\fish\dev\out\pipette


In [4]:
frames = fish.cached_read(DATA / "D1-1.hsv")
frames.shape

DEBUG:fish.io:Cached read from D:\projects\fish\data\D1-1.hsv.cached


(1816, 1024, 1280)

In [ ]:
bgnd = fish.background_via_min(frames)

In [ ]:
dish = fish.find_dish(bgnd)
mask = dish.mask_like(bgnd)
fish.show_frame(mask * 255)

In [ ]:
def label_largest_area(frame):
    without_bgnd = np.where(bgnd < frame, frame - bgnd, 0)
    masked = fish.apply_mask(without_bgnd, mask)
    thresh, thresholded = cv.threshold(masked, thresh = 0, maxval = 255, type = cv.THRESH_OTSU)
    
    with_edge = dish.draw_on(thresholded)
    inverted = cv.bitwise_not(with_edge)
    
    h, w = inverted.shape
    flood_mask = np.zeros((h + 2, w + 2), dtype = np.uint8)
    flood_mask[1:-1, 1:-1] = inverted
    
    _, flooded, b, _ = cv.floodFill(inverted, mask = flood_mask, seedPoint = (dish.x + dish.r, dish.y),
                                    newVal = 127,  flags = 8)
    
    # return flooded
    thingy = np.where(flooded == 127, 1, 0).astype(np.uint8)
    area = np.sum(thingy)
    
    return area
    
# fish.show_frame(label_largest_area(frames[30]))

In [ ]:
areas = [label_largest_area(frame) for frame in tqdm(frames)]

In [ ]:
plt.close()
fig = plt.Figure(figsize = (6, 4), dpi = 300)
ax = fig.add_subplot(111)
ax.plot(areas)

fig

In [ ]:
areas = np.array(areas)
np.sum(areas > 6000)

plt.close()
fig = plt.Figure(figsize = (6, 4), dpi = 300)
ax = fig.add_subplot(111)
ax.plot(areas > 6000)

ax.set_xlim(0, 300)

fig

In [ ]:
# a = np.expand_dims(areas, 1)
# print(a.shape)
labels, num_features = ndi.measurements.label(areas > 6000)
print(num_features)
slices = ndi.measurements.find_objects(labels)
print(slices)
slices = [s[0] for s in slices]
pipette_slice = max(slices, key = lambda s: s.stop - s.start)
print(pipette_slice)
start_frame = pipette_slice.stop + 10

In [ ]:
f = frames[:50]
fish.make_movie(OUT / "test.mp4", (label_largest_area(frame) for frame in f), num_frames = len(f), fps = 2)

In [5]:
fish.find_last_pipette_frame(frames)

Calculating area of objects touching the disk...: 100%|██████████| 1816/1816 [00:51<00:00, 35.52it/s]


44